In [12]:
import os
import gc
import glob
import json
import datetime
from collections import defaultdict
import multiprocessing as mp
from pathlib import Path
from types import SimpleNamespace
from typing import Dict, List, Optional, Tuple
import warnings

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from tqdm import tqdm
import tifffile
import random
import math

In [13]:
import logging
import logging.handlers

root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)

# Some libraries attempt to add their own root logger handlers. This is
# annoying and so we get rid of them.
for handler in list(root_logger.handlers):
    root_logger.removeHandler(handler)

logfmt_str = "%(asctime)s %(levelname)-8s pid:%(process)d %(name)s:%(lineno)03d:%(funcName)s %(message)s"
formatter = logging.Formatter(logfmt_str)

streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
streamHandler.setLevel(logging.DEBUG)

root_logger.addHandler(streamHandler)

In [14]:
from google.colab import drive

In [15]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# def create_bounding_box(image):
#     # Find the rows and columns where pixel values are equal to 1
#     rows = np.any(image == 1, axis=1)
#     cols = np.any(image == 1, axis=0)
    
#     # Find the indices of the first and last non-zero rows and columns
#     rmin, rmax = np.where(rows)[0][[0, -1]]
#     cmin, cmax = np.where(cols)[0][[0, -1]]
    
#     # Create the bounding box
#     bounding_box = (rmin, rmax+1, cmin, cmax+1)
    
#     return bounding_box



# def create_mask(image, bounding_box):
#     # Create an empty mask with the same shape as the input image
#     mask = np.zeros_like(image, dtype=np.bool)
    
#     # Unpack the bounding box coordinates
#     rmin, rmax, cmin, cmax = bounding_box
    
#     # Set the values inside the bounding box to True
#     mask[rmin:rmax, cmin:cmax] = True
    
#     return mask

In [17]:
class Ind_ID:
    def __init__(self, mode='test', id=None, size=None):

        assert mode in ['train', 'test'], "mode must be either 'train' or 'test'"

        self.id = str(id)
        self.mode = mode
        self.size = size if isinstance(size, tuple) else (size, size)
        self.pos_label = None
        
        id_path = glob.glob('/content/drive/MyDrive/Data/Vesuvisius/{}/{}'.format(self.mode, self.id))[0]
        print(id_path)

        image_paths = sorted(glob.glob(os.path.join(id_path, 'surface_volume', '*.tif')))
        images = []
        for image_path in image_paths:
            image = tifffile.imread(image_path)
            
            if self.size:
                image = self.resize_image(image, self.size).astype(np.float32)
            else:
                image = image.astype(np.float32)
                
            images.append(image)
        self.stacked = np.stack(images, axis=0)
        
        self.pos_mask = self.create_mask(self.stacked)
        
        # self.pos_mask = np.expand_dims(self.pos_mask, axis=0)
        # self.pos_mask = np.repeat(self.pos_mask, self.stacked.shape[0], axis=0)
        # print(self.pos_mask)
        self.positive_indexes = (self.pos_mask.sum(axis=(1,2))
                                 .nonzero()[0].tolist())
        print(self.positive_indexes)
        if self.mode == 'train':
            label_path = os.path.join(id_path, 'inklabels.png')
            assert os.path.exists(label_path), "inklabels.png does not exist in the folder"

            self.pos_label = self.load_label(label_path)
            print(self.pos_label)
            self.pos_label = np.expand_dims(self.pos_label, axis=0)
            self.pos_label = np.repeat(self.pos_label, self.stacked.shape[0], axis=0)
            # print(self.pos_label.shape)
            self.positive_indexes_lbl = (self.pos_label.sum(axis=(1,2)).nonzero()[0].tolist())
        # else:
        #     self.pos_label = None
        #     self.positive_indexes_lbl = None
        #     print("Warning: 'inklabel.png' file will not be used in the 'test' mode")
        
    def resize_image(self, image, size):
        # Convert the image to a supported data type
        image = image.astype(np.uint8)
        # Convert the image to a PIL Image
        pil_image = Image.fromarray(image)
        # Resize the image using the specified size
        pil_image = pil_image.resize(size, Image.ANTIALIAS)
        # Convert the PIL Image back to a numpy array
        image = np.array(pil_image, dtype=bool)
        return image

    def load_label(self, label_path):
        label = Image.open(label_path)

        binary_label = label.convert('1')

        # Convert the binary mask to a numpy array
        binary_label = np.array(binary_label).astype(int)

        # Convert the values to [False, True]
        binary_label = binary_label.astype(bool)

        if self.size:
            binary_label = self.resize_image(binary_label, self.size)
        else:
            binary_label = np.array(binary_label).astype(np.float32)

        return binary_label

    def load_mask(self, path):
        # id_path = glob.glob('/content/drive/MyDrive/Data/Vesuvisius/{}/{}'.format(self.mode, self.id))[0]

        mask = Image.open(self.path)

        binary_mask = mask.convert('1')

        # Convert the binary mask to a numpy array
        binary_mask = np.array(binary_mask).astype(bool)
        
        if self.size:
            binary_mask = self.resize_image(binary_mask, self.size)
        else:
            binary_mask = np.array(binary_mask).astype(np.float32)
        
        return binary_mask


    def create_mask(self, image):
        # Find the rows and columns where pixel values are equal to 1
        rows = np.any(image == 1, axis=1)
        cols = np.any(image == 1, axis=0)

        # Find the indices of the first and last non-zero rows and columns
        rmin, rmax = np.where(rows)[0][[0, -1]]
        cmin, cmax = np.where(cols)[0][[0, -1]]

        # Create an empty mask with the same shape as the input image
        mask = np.zeros_like(image, dtype='bool')

        # Set the values inside the bounding box to True
        mask[rmin:rmax+1, cmin:cmax+1] = True
        
        return mask

    def get_raw_candidate(self, index):
        # Get the image and mask at the specified index
        image = self.stacked[index]
        mask = self.pos_mask[index]

        # if hasattr(self, 'pos_label'):
        label = self.pos_label[index]
        return image, mask, label
        # else:
        #     return image, mask


print(Ind_ID('train', '1', (512, 512)))


/content/drive/MyDrive/Data/Vesuvisius/train/1
[0, 1, 2, 3, 4]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [18]:
def get_Id_list(mode):
    path = '/content/drive/MyDrive/Data/Vesuvisius/{}'.format(mode)
    files = os.listdir(path)
    return files

def getCt(mode, id, size):
    return Ind_ID(mode, id, size)


def getSampleSize(mode, id, size):
    ct = Ind_ID(mode, id, size)
    return int(ct.stacked.shape[0]), ct.positive_indexes

print(getSampleSize('train', '1', (512, 512)))

/content/drive/MyDrive/Data/Vesuvisius/train/1
[0, 1, 2, 3, 4]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
(5, [0, 1, 2, 3, 4])


In [19]:
import cv2

In [55]:
class ValidationSegmentationDataset(Dataset):
    def __init__(self, mode='test', id=None, size=None, context_slices = 5, full_bool = False):
        assert mode in ['train', 'test'], "mode must be either 'train' or 'test'"
        
        self.mode = mode
        self.context_slices = context_slices
        self.full_bool = full_bool
        self.size = size

        # if isinstance(id, int):
        #     self.id = str(id)
        # else:
        #     self.id = id

        if id:
          self.id_list = [id]
        else:
          self.id_list = sorted(get_Id_list(self.mode))

        if self.mode == 'train':
          self.id_list = sorted(get_Id_list(self.mode))
        elif self.mode == 'test':
          self.id_list = sorted(get_Id_list(self.mode))


        
        self.sample_list = []

        for id in self.id_list:
          index_ct, pos_indx = getSampleSize(self.mode, id, self.size)

          if self.full_bool:
                self.sample_list += [(id, slice_ndx)
                                     for slice_ndx in range(index_ct)]
          else:
                self.sample_list += [(id, slice_ndx)
                                     for slice_ndx in pos_indx]
        # print(self.id_list)
        # print(self.sample_list)
    
    def __len__(self):
        return len(self.sample_list)

    def __getitem__(self, ndx):
        series_uid, slice_ndx = self.sample_list[ndx % len(self.sample_list)]
        print("wow", series_uid, slice_ndx)
        return self.getitem_fullSlice(series_uid, slice_ndx)

    def getitem_fullSlice(self, id, slice_ndx):
        ct = getCt(self.mode, id, self.size)
        ct_t = torch.zeros((self.context_slices * 1, 512, 512))

        start_ndx = (slice_ndx - self.context_slices) *(-1)
        end_ndx = slice_ndx + self.context_slices + 1
        print((start_ndx, end_ndx))

        for i, context_ndx in enumerate(range(start_ndx, end_ndx)):
            context_ndx = max(context_ndx, 0)
            context_ndx = min(context_ndx, ct.stacked.shape[0] - 1)
            # ct_t[i] = torch.from_numpy(ct.stacked[context_ndx].astype(np.float32))

            # Check the shape of the stacked attribute
            stacked_shape = ct.stacked[context_ndx].shape
            if stacked_shape != (512, 512):
                # Resize the stacked attribute if necessary
                stacked_resized = cv2.resize(ct.stacked[context_ndx], (512, 512))
                # print("&&******&&&&&&&", stacked_resized.shape)

                ct_t[i] = torch.from_numpy(stacked_resized.astype(np.float32))
            else:
                print("&&******&&&&&&&", ct.stacked[context_ndx].shape)
                ct_t[i] = torch.from_numpy(ct.stacked[context_ndx].astype(np.float32))

        pos_t = torch.from_numpy(ct.pos_mask[slice_ndx]).unsqueeze(0)
        if ct.pos_label is not None:
            ink_t = torch.from_numpy(ct.pos_label[slice_ndx]).unsqueeze(0)
        else:
            ink_t = None

        if ink_t is not None:
            return ct_t, pos_t, ink_t, ct.id, slice_ndx
        else:
            return ct_t, pos_t, ct.id, slice_ndx
        
    

class TrainingSegmentationDataset(ValidationSegmentationDataset):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # self.size = (256, 256)


    def __getitem__(self, ndx):
        series_uid, slice_ndx = self.sample_list[ndx % len(self.sample_list)]
        
        return self.getitem_trainCrop(series_uid, slice_ndx)  
    
    def getitem_trainCrop(self, id, slice_ndx):
        ct = getCt(self.mode, id, self.size)
        ct_t = torch.zeros((self.context_slices * 2 - 1, 256, 256))
        
        start_ndx = slice_ndx - self.context_slices
        end_ndx = start_ndx + 5
        
        for i, context_ndx in enumerate(range(start_ndx+1, end_ndx+1)):
            context_ndx = max(context_ndx, 0)
            context_ndx = min(context_ndx, ct.stacked.shape[0] - 1)

            stacked_shape = ct.stacked[context_ndx].shape
            if stacked_shape != (256, 256):
                stacked_resized = cv2.resize(ct.stacked[context_ndx], (256, 256))
                print("stacked_resized ", stacked_resized.shape)
                ct_t[i] = torch.from_numpy(stacked_resized.astype(np.float32))
            else:
                ct_t[i] = torch.from_numpy(ct.stacked[context_ndx].astype(np.float32))

        pos_t = torch.from_numpy(ct.pos_mask[slice_ndx].astype(np.float32)).unsqueeze(0)
        
        if ct.pos_label is not None:
            ink_t = torch.from_numpy(ct.pos_label[slice_ndx].astype(np.float32)).unsqueeze(0)
        else:
            ink_t = None

        if ink_t is not None:
            return ct_t, pos_t, ink_t, ct.id, slice_ndx
        else:
            return ct_t, pos_t, ct.id, slice_ndx

    
        



In [45]:
data_dir = "/content/drive/MyDrive/Data/Vesuvisius"

augmentation_dict = {'flip': True, 'offset': 0.1, 'scale': 0.2, 'rotate': True, 'noise': 25.0}
# Create a SegmentationDataset object
# test_dataset= ValidationSegmentationDataset('test', 'a', (512, 512), 5, True)
# print(test_dataset[0][0].shape)
train_dataset = TrainingSegmentationDataset('train', '1', (256, 256), 5, True)
print(train_dataset[0][0].shape)

# for i in train_dataset:
#   print(i)


# Use the dataset object to access a fragment
# fragment = dataset[image][0][1]
# print(fragment.shape)
# Use indexing to access an individual image within the fragment
# image = fragment
# print(torch.max(image).item())
# mask = image > 0

# # Use the mask to index the image tensor and get the elements that are greater than 0
# image_gt_0 = image[mask]

# # Print the resulting tensor
# print(image_gt_0)

# Display the image using a library such as matplotlib
# import matplotlib.pyplot as plt
# plt.imshow(image, cmap='gray')
# plt.show()

/content/drive/MyDrive/Data/Vesuvisius/train/1
[0, 1, 2, 3, 4]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
/content/drive/MyDrive/Data/Vesuvisius/train/2
[0, 1, 2, 3, 4]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
/content/drive/MyDrive/Data/Vesuvisius/train/3
[0, 1, 2, 3, 4]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
/content/drive/MyDr

In [46]:
# for i, sample in enumerate(dataset):
#             print(f"Sample {i}: {sample}")

In [47]:
# labels, inklabels = dataset[0][2]
# print(torch.min(inklabels).item())
# plt.imshow(inklabels, cmap='gray')
# plt.show()


In [48]:
import torch
from torch import nn
import torch.nn.functional as F


class UNet(nn.Module):
    def __init__(self, in_channels=1, n_classes=2, depth=5, wf=6, padding=False,
                 batch_norm=False, up_mode='upconv'):
        """
        Implementation of
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        (Ronneberger et al., 2015)
        https://arxiv.org/abs/1505.04597
        Using the default arguments will yield the exact version used
        in the original paper
        Args:
            in_channels (int): number of input channels
            n_classes (int): number of output channels
            depth (int): depth of the network
            wf (int): number of filters in the first layer is 2**wf
            padding (bool): if True, apply padding such that the input shape
                            is the same as the output.
                            This may introduce artifacts
            batch_norm (bool): Use BatchNorm after layers with an
                               activation function
            up_mode (str): one of 'upconv' or 'upsample'.
                           'upconv' will use transposed convolutions for
                           learned upsampling.
                           'upsample' will use bilinear upsampling.
        """
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(UNetConvBlock(prev_channels, 2**(wf+i),
                                                padding, batch_norm))
            prev_channels = 2**(wf+i)

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth - 1)):
            self.up_path.append(UNetUpBlock(prev_channels, 2**(wf+i), up_mode,
                                            padding, batch_norm))
            prev_channels = 2**(wf+i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path)-1:
                blocks.append(x)
                x = F.avg_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i-1])

        return self.last(x)


class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3,
                               padding=int(padding)))
        block.append(nn.ReLU())
        # block.append(nn.LeakyReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3,
                               padding=int(padding)))
        block.append(nn.ReLU())
        # block.append(nn.LeakyReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out


class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2,
                                         stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(nn.Upsample(mode='bilinear', scale_factor=2),
                                    nn.Conv2d(in_size, out_size, kernel_size=1))

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[:, :, diff_y:(diff_y + target_size[0]), diff_x:(diff_x + target_size[1])]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

In [49]:
import math
import random
from collections import namedtuple

import torch
from torch import nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
# from util.logconf import logging
# from util.unet import UNet

log = logging.getLogger(__name__)
log.setLevel(logging.WARN)
log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

class UNetWrapper(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

        self.input_batchnorm = nn.BatchNorm2d(kwargs['in_channels'])
        self.unet = UNet(**kwargs)
        self.final = nn.Sigmoid()

        self._init_weights()

    def _init_weights(self):
        init_set = {
            nn.Conv2d,
            nn.Conv3d,
            nn.ConvTranspose2d,
            nn.ConvTranspose3d,
            nn.Linear,
        }
        for m in self.modules():
            if type(m) in init_set:
                nn.init.kaiming_normal_(
                    m.weight.data, mode='fan_out', nonlinearity='relu', a=0
                )
                if m.bias is not None:
                    fan_in, fan_out = \
                        nn.init._calculate_fan_in_and_fan_out(m.weight.data)
                    bound = 1 / math.sqrt(fan_out)
                    nn.init.normal_(m.bias, -bound, bound)

        # nn.init.constant_(self.unet.last.bias, -4)
        # nn.init.constant_(self.unet.last.bias, 4)


    def forward(self, input_batch):
        bn_output = self.input_batchnorm(input_batch)
        un_output = self.unet(bn_output)
        fn_output = self.final(un_output)
        return fn_output

class SegmentationAugmentation(nn.Module):
    def __init__(
            self, flip=None, offset=None, scale=None, rotate=None, noise=None
    ):
        super().__init__()

        self.flip = flip
        self.offset = offset
        self.scale = scale
        self.rotate = rotate
        self.noise = noise

    def forward(self, input_g, label_g, inklabel_g=None):
        transform_t = self._build2dTransformMatrix()
        transform_t = transform_t.expand(input_g.shape[0], -1, -1)
        transform_t = transform_t.to(input_g.device, torch.float32)
        affine_t = F.affine_grid(transform_t[:,:2],
                input_g.size(), align_corners=False)

        augmented_input_g = F.grid_sample(input_g,
                affine_t, padding_mode='border',
                align_corners=False)
        augmented_label_g = F.grid_sample(label_g.to(torch.float32),
                affine_t, padding_mode='border',
                align_corners=False)
        if inklabel_g is not None:
            augmented_inklabel_g = F.grid_sample(inklabel_g.to(torch.float32),
                affine_t, padding_mode='border', 
                align_corners=False)
        else:
            augmented_inklabel_g = None

        if self.noise:
            noise_t = torch.randn_like(augmented_input_g)
            noise_t *= self.noise

            augmented_input_g += noise_t

        return augmented_input_g, augmented_label_g > 0.5, augmented_inklabel_g > 0.5 if inklabel_g is not None else None


    def _build2dTransformMatrix(self):
        transform_t = torch.eye(3)

        for i in range(2):
            if self.flip:
                if random.random() > 0.5:
                    transform_t[i,i] *= -1

            if self.offset:
                offset_float = self.offset
                random_float = (random.random() * 2 - 1)
                transform_t[2,i] = offset_float * random_float

            if self.scale:
                scale_float = self.scale
                random_float = (random.random() * 2 - 1)
                transform_t[i,i] *= 1.0 + scale_float * random_float

        if self.rotate:
            angle_rad = random.random() * math.pi * 2
            s = math.sin(angle_rad)
            c = math.cos(angle_rad)

            rotation_t = torch.tensor([
                [c, -s, 0],
                [s, c, 0],
                [0, 0, 1]])

            transform_t @= rotation_t

        return transform_t

In [50]:
# im = dataset[0][1].unsqueeze(0)
# label = dataset[0][2].unsqueeze(0)
# ink = dataset[0][3].unsqueeze(0)

# a, b, c = segmentation_augmentation(im, label, ink, flip=True, offset=0.03, scale=0.2, rotate=True, noise=25.0)

# # import matplotlib.pyplot as plt
# image_np = a.squeeze(0).numpy()

# fig, axs = plt.subplots(1, 3)
# for i in range(3):
#     axs[i].imshow(image_np[i], cmap='gray')
# plt.show()

In [51]:
import collections
import copy
import datetime
import gc
import time

# import torch
import numpy as np

# from util.logconf import logging
# log = logging.getLogger(__name__)
# # log.setLevel(logging.WARN)
# # log.setLevel(logging.INFO)
# log.setLevel(logging.DEBUG)


def enumerateWithEstimate(
        iter,
        desc_str,
        start_ndx=0,
        print_ndx=4,
        backoff=None,
        iter_len=None,
):
    
    if iter_len is None:
        iter_len = len(iter)

    if backoff is None:
        backoff = 2
        while backoff ** 7 < iter_len:
            backoff *= 2

    assert backoff >= 2
    while print_ndx < start_ndx * backoff:
        print_ndx *= backoff

    log.warning("{} ----/{}, starting".format(
        desc_str,
        iter_len,
    ))
    start_ts = time.time()
    for (current_ndx, item) in enumerate(iter):
        yield (current_ndx, item)
        if current_ndx == print_ndx:
            # ... <1>
            duration_sec = ((time.time() - start_ts)
                            / (current_ndx - start_ndx + 1)
                            * (iter_len-start_ndx)
                            )

            done_dt = datetime.datetime.fromtimestamp(start_ts + duration_sec)
            done_td = datetime.timedelta(seconds=duration_sec)

            log.info("{} {:-4}/{}, done at {}, {}".format(
                desc_str,
                current_ndx,
                iter_len,
                str(done_dt).rsplit('.', 1)[0],
                str(done_td).rsplit('.', 1)[0],
            ))

            print_ndx *= backoff

        if current_ndx + 1 == start_ndx:
            start_ts = time.time()

    log.warning("{} ----/{}, done at {}".format(
        desc_str,
        iter_len,
        str(datetime.datetime.now()).rsplit('.', 1)[0],
    ))

In [52]:
# for i, sample in enumerateWithEstimate(train_dataset, "Processing samples"):
#     print(f"Sample {i}:")
#     print(f"  ID: {sample[0]}")
#     print(f"  Image shape: {sample[1].shape}")
#     print(f"  Label shape: {sample[2][0]}")

In [53]:
import hashlib
import os
import shutil
import socket
import sys

import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.optim

from torch.optim import SGD, Adam
from torch.utils.data import DataLoader

# from util.util import enumerateWithEstimate
# from .dsets import Luna2dSegmentationDataset, TrainingLuna2dSegmentationDataset, getCt
# from util.logconf import logging
# from .model import UNetWrapper, SegmentationAugmentation

log = logging.getLogger(__name__)
# log.setLevel(logging.WARN)
# log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

# Used for computeClassificationLoss and logMetrics to index into metrics_t/metrics_a
# METRICS_LABEL_NDX = 0
METRICS_LOSS_NDX = 1
METRICS_TP_NDX = 7
METRICS_FN_NDX = 8
METRICS_FP_NDX = 9

METRICS_SIZE = 10

class SegmentationTrainingApp:
    def __init__(self, augmentation_dict=None):
        if augmentation_dict is None:
            self.augmentation_dict = {
             'flip': True,
              'offset': 0.1,
              'scale': 0.2,
              'rotate': True, 
              'noise': 25.0
             }
        else:
            self.augmentation_dict = augmentation_dict

        self.time_str = datetime.datetime.now().strftime('%Y-%m-%d_%H.%M.%S')
        self.totalTrainingSamples_count = 0
        self.trn_writer = None
        self.val_writer = None
        self.epochs = 1
        self.batch_size = 4

        # self.augmentation_dict = self.augmentation_options

        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        self.segmentation_model, self.augmentation_model = self.initModel()
        self.optimizer = self.initOptimizer()

    def initModel(self):
        segmentation_model = UNetWrapper(
            in_channels=3,
            n_classes=1,
            depth=3,
            wf=4,
            padding=True,
            batch_norm=True,
            up_mode='upconv',
        )

        augmentation_model = SegmentationAugmentation(**self.augmentation_dict)

        if self.use_cuda:
            log.info("Using CUDA; {} devices.".format(torch.cuda.device_count()))
            if torch.cuda.device_count() > 1:
                segmentation_model = nn.DataParallel(segmentation_model)
                augmentation_model = nn.DataParallel(augmentation_model)
            segmentation_model = segmentation_model.to(self.device)
            augmentation_model = augmentation_model.to(self.device)

        return segmentation_model, augmentation_model

    def initOptimizer(self):
        return Adam(self.segmentation_model.parameters())

    def initTrainDl(self):
        train_ds = TrainingSegmentationDataset('train', (256, 256), 3, True)

        batch_size = 4
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        train_dl = DataLoader(
            train_ds,
            batch_size=batch_size,
            num_workers=4,
            pin_memory=self.use_cuda,
        )

        return train_dl

    def initTestDl(self):
        test_ds = ValidationSegmentationDataset('test', (512, 512), 3, True)

        batch_size = 4
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        test_dl = DataLoader(
            test_ds,
            batch_size=batch_size,
            num_workers=4,
            pin_memory=self.use_cuda,
        )

        return test_dl


    def main(self):
        # log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        test_dl = self.initTestDl() # remove validation dataset
        # print(train_dl.dataset[0][1].shape)

        best_score = 0.0
        # self.validation_cadence = 5 # remove validation cadence
        for epoch_ndx in range(1, self.epochs + 1):
            log.info("Epoch {} of {}, {} batches of size {}*{}".format(
                epoch_ndx,
                self.epochs,
                len(train_dl),
                len(test_dl), # remove validation dataset
                self.batch_size,
                (torch.cuda.device_count() if self.use_cuda else 1),
            ))

        trnMetrics_t = self.doTraining(epoch_ndx, train_dl)
        self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)

            # if epoch_ndx == 1 or epoch_ndx % self.validation_cadence == 0:
            #     # if validation is wanted
            #     valMetrics_t = self.doTesting(epoch_ndx, val_dl)
            #     score = self.logMetrics(epoch_ndx, 'val', valMetrics_t)
            #     best_score = max(score, best_score)

            # self.saveModel('seg', epoch_ndx, score == best_score)

            # self.logImages(epoch_ndx, 'trn', train_dl)
            # self.logImages(epoch_ndx, 'val', val_dl)

        self.trn_writer.close()
        # self.val_writer.close()

    def doTraining(self, epoch_ndx, train_dl):
        # print(len(train_dl.dataset))
        trnMetrics_g = torch.zeros(METRICS_SIZE, len(train_dl.dataset), device=self.device)
        self.segmentation_model.train()
        # train_dl.dataset.shuffleSamples()

        batch_iter = enumerateWithEstimate(
            train_dl,
            "E{} Training".format(epoch_ndx),
            start_ndx=train_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.optimizer.zero_grad()

            loss_var = self.computeBatchLoss(batch_ndx, batch_tup, train_dl.batch_size, trnMetrics_g)
            loss_var.backward()

            self.optimizer.step()

        self.totalTrainingSamples_count += trnMetrics_g.size(1)

        return trnMetrics_g.to('cpu')

    def doTesting(self, epoch_ndx, test_dl):
        with torch.no_grad():
            testMetrics_g = torch.zeros(METRICS_SIZE, len(test_dl.dataset), device=self.device)
            self.segmentation_model.eval()

            batch_iter = enumerateWithEstimate(
                test_dl,
                "E{} Testing ".format(epoch_ndx),
                start_ndx=test_dl.num_workers,
            )
            for batch_ndx, batch_tup in batch_iter:
                self.computeBatchLoss(batch_ndx, batch_tup, test_dl.batch_size, testMetrics_g)

        return testMetrics_g.to('cpu')

    def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g,
                         classificationThreshold=0.5):
        input_t, label_t, ink_t, series_list, _slice_ndx_list = batch_tup

        input_g = input_t.to(self.device, non_blocking=True)
        label_g = label_t.to(self.device, non_blocking=True)
        ink_g = ink_t.to(self.device, non_blocking=True)

        if self.segmentation_model.training and self.augmentation_dict:
            input_g, label_g, ink_g = self.augmentation_model(input_g, label_g, ink_g)

        prediction_g = self.segmentation_model(input_g)
        combined_label_mask_t = label_g | ink_g
        diceLoss_g = self.diceLoss(prediction_g, combined_label_mask_t)
        fnLoss_g = self.diceLoss(prediction_g * combined_label_mask_t, combined_label_mask_t)

        start_ndx = batch_ndx * batch_size
        end_ndx = start_ndx + input_t.size(0)

        with torch.no_grad():
            predictionBool_g = (prediction_g[:, 0:1]
                                > classificationThreshold).to(torch.float32)

            tp = (     predictionBool_g *  label_g).sum(dim=[1,2,3])
            fn = ((1 - predictionBool_g) *  label_g).sum(dim=[1,2,3])
            fp = (     predictionBool_g * (~label_g)).sum(dim=[1,2,3])

            metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = diceLoss_g
            metrics_g[METRICS_TP_NDX, start_ndx:end_ndx] = tp
            metrics_g[METRICS_FN_NDX, start_ndx:end_ndx] = fn
            metrics_g[METRICS_FP_NDX, start_ndx:end_ndx] = fp

        return diceLoss_g.mean() + fnLoss_g.mean() * 8

    def diceLoss(self, prediction_g, label_g, epsilon=1):
        diceLabel_g = label_g.sum(dim=[1,2,3])
        dicePrediction_g = prediction_g.sum(dim=[1,2,3])
        diceCorrect_g = (prediction_g * label_g).sum(dim=[1,2,3])

        diceRatio_g = (2 * diceCorrect_g + epsilon) \
            / (dicePrediction_g + diceLabel_g + epsilon)

        return 1 - diceRatio_g

    # def logImages(self, epoch_ndx, mode_str, dl):
    #     self.segmentation_model.eval()

    #     images = sorted(dl.dataset.series_list)[:12]
    #     for series_ndx, series_uid in enumerate(images):
    #         ct = getCt(series_uid)

    #         for slice_ndx in range(6):
    #             ct_ndx = slice_ndx * (ct.hu_a.shape[0] - 1) // 5
    #             sample_tup = dl.dataset.getitem_fullSlice(series_uid, ct_ndx)

    #             ct_t, label_t, series_uid, ct_ndx = sample_tup

    #             input_g = ct_t.to(self.device).unsqueeze(0)
    #             label_g = pos_g = label_t.to(self.device).unsqueeze(0)

    #             prediction_g = self.segmentation_model(input_g)[0]
    #             prediction_a = prediction_g.to('cpu').detach().numpy()[0] > 0.5
    #             label_a = label_g.cpu().numpy()[0][0] > 0.5

    #             ct_t[:-1,:,:] /= 2000
    #             ct_t[:-1,:,:] += 0.5

    #             ctSlice_a = ct_t[dl.dataset.contextSlices_count].numpy()

    #             image_a = np.zeros((512, 512, 3), dtype=np.float32)
    #             image_a[:,:,:] = ctSlice_a.reshape((512,512,1))
    #             image_a[:,:,0] += prediction_a & (1 - label_a)
    #             image_a[:,:,0] += (1 - prediction_a) & label_a
    #             image_a[:,:,1] += ((1 - prediction_a) & label_a) * 0.5

    #             image_a[:,:,1] += prediction_a & label_a
    #             image_a *= 0.5
    #             image_a.clip(0, 1, image_a)

    #             writer = getattr(self, mode_str + '_writer')
    #             writer.add_image(
    #                 f'{mode_str}/{series_ndx}_prediction_{slice_ndx}',
    #                 image_a,
    #                 self.totalTrainingSamples_count,
    #                 dataformats='HWC',
    #             )

    #             if epoch_ndx == 1:
    #                 image_a = np.zeros((512, 512, 3), dtype=np.float32)
    #                 image_a[:,:,:] = ctSlice_a.reshape((512,512,1))
    #                 # image_a[:,:,0] += (1 - label_a) & lung_a # Red
    #                 image_a[:,:,1] += label_a  # Green
    #                 # image_a[:,:,2] += neg_a  # Blue

    #                 image_a *= 0.5
    #                 image_a[image_a < 0] = 0
    #                 image_a[image_a > 1] = 1
    #                 writer.add_image(
    #                     '{}/{}_label_{}'.format(
    #                         mode_str,
    #                         series_ndx,
    #                         slice_ndx,
    #                     ),
    #                     image_a,
    #                     self.totalTrainingSamples_count,
    #                     dataformats='HWC',
    #                 )
    #             # This flush prevents TB from getting confused about which
    #             # data item belongs where.
    #             writer.flush()

    def logMetrics(self, epoch_ndx, mode_str, metrics_t):
        log.info("E{} {}".format(
            epoch_ndx,
            type(self).__name__,
        ))

        metrics_a = metrics_t.detach().numpy()
        sum_a = metrics_a.sum(axis=1)
        assert np.isfinite(metrics_a).all()

        allLabel_count = sum_a[METRICS_TP_NDX] + sum_a[METRICS_FN_NDX]

        metrics_dict = {}
        metrics_dict['loss/all'] = metrics_a[METRICS_LOSS_NDX].mean()

        metrics_dict['percent_all/tp'] = \
            sum_a[METRICS_TP_NDX] / (allLabel_count or 1) * 100
        metrics_dict['percent_all/fn'] = \
            sum_a[METRICS_FN_NDX] / (allLabel_count or 1) * 100
        metrics_dict['percent_all/fp'] = \
            sum_a[METRICS_FP_NDX] / (allLabel_count or 1) * 100


        precision = metrics_dict['pr/precision'] = sum_a[METRICS_TP_NDX] \
            / ((sum_a[METRICS_TP_NDX] + sum_a[METRICS_FP_NDX]) or 1)
        recall    = metrics_dict['pr/recall']    = sum_a[METRICS_TP_NDX] \
            / ((sum_a[METRICS_TP_NDX] + sum_a[METRICS_FN_NDX]) or 1)

        metrics_dict['pr/f1_score'] = 2 * (precision * recall) \
            / ((precision + recall) or 1)

        log.info(("E{} {:8} "
                 + "{loss/all:.4f} loss, "
                 + "{pr/precision:.4f} precision, "
                 + "{pr/recall:.4f} recall, "
                 + "{pr/f1_score:.4f} f1 score"
                  ).format(
            epoch_ndx,
            mode_str,
            **metrics_dict,
        ))
        log.info(("E{} {:8} "
                  + "{loss/all:.4f} loss, "
                  + "{percent_all/tp:-5.1f}% tp, {percent_all/fn:-5.1f}% fn, {percent_all/fp:-9.1f}% fp"
        ).format(
            epoch_ndx,
            mode_str + '_all',
            **metrics_dict,
        ))

        # self.initTensorboardWriters()
        writer = getattr(self, mode_str + '_writer')

        prefix_str = 'seg_'
        print(metrics_dict)
        # for key, value in metrics_dict.items():
        #     writer.add_scalar(prefix_str + key, value, self.totalTrainingSamples_count)

        # writer.flush()

        score = metrics_dict['pr/recall']

        return score




In [54]:
if __name__ == '__main__':
    SegmentationTrainingApp().main()

/content/drive/MyDrive/Data/Vesuvisius/train/1
[0, 1, 2, 3, 4]
[[False False False]
 [False False False]
 [False False False]]
/content/drive/MyDrive/Data/Vesuvisius/train/2
[0, 1, 2, 3, 4]
[[False False False]
 [False False False]
 [False False False]]
/content/drive/MyDrive/Data/Vesuvisius/train/3
[0, 1, 2, 3, 4]
[[False False False]
 [False False False]
 [False False False]]
/content/drive/MyDrive/Data/Vesuvisius/test/a


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[0, 1, 2, 3, 4]
/content/drive/MyDrive/Data/Vesuvisius/test/b


2023-06-15 03:27:24,831 INFO     pid:532 __main__:133:main Epoch 1 of 1, 4 batches of size 3*4
2023-06-15 03:27:24,834 WARNING  pid:532 __main__:038:enumerateWithEstimate E1 Training ----/4, starting


[0, 1, 2, 3, 4]
/content/drive/MyDrive/Data/Vesuvisius/train/1
/content/drive/MyDrive/Data/Vesuvisius/train/1/content/drive/MyDrive/Data/Vesuvisius/train/2

/content/drive/MyDrive/Data/Vesuvisius/train/3
[0, 1, 2, 3, 4]
[[False False False]
 [False False False]
 [False False False]]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[[False False False]
 [False False False]
 [False False False]]


RuntimeError: ignored

[[False False False]
 [False False False]
 [False False False]]
